In [1]:
import pandas as pd
import numpy as np
from sklearn import metrics
# from sklearn.naive_bayes import GaussianNB
# from sklearn.naive_bayes import CategoricalNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

### Handle nan sentences

In [2]:
domain_1 = pd.read_csv('domain_1_set_clean.csv', sep='\t', encoding='utf-8', usecols=['sentence', 'annotation'])
domain_2 = pd.read_csv('domain_2_set_clean.csv', sep='\t', encoding='utf-8', usecols=['sentence', 'annotation'])

In [3]:
domain_1

,sentence,annotation
0,model monthly notizie scritte cost around bill...,non_argument
1,trumpets constructed brass,non_argument
2,according proponents policies uniforms improve...,argument
3,cantons abrahamic religions judaism christiani...,non_argument
4,historians sympathized millau viaduct worlds p...,non_argument
...,...,...
12095,cooling operations often requires whole lot wa...,argument
12096,connect cheap offwhite paper known rhône valle...,non_argument
12097,continent equal zero chemical formula often,non_argument
12098,meant view penrose discusses long career also ...,non_argument


In [4]:
# drop nan sentences
domain_1 = domain_1[domain_1['sentence'].notna()]

In [5]:
# drop two arguments and two non-arguments
domain_1 = domain_1.drop(domain_1.index[[0,1,2,12095]])

In [6]:
domain_1["annotation"].value_counts()

non_argument    6048
argument        6048
Name: annotation, dtype: int64

In [7]:
len(domain_1)

12096

In [8]:
# drop nan sentences
domain_2 = domain_2[domain_2['sentence'].notna()]

In [9]:
# drop two arguments
domain_2 = domain_2.drop(domain_2.index[[0,3]])

In [10]:
domain_2["annotation"].value_counts()

non_argument    6048
argument        6048
Name: annotation, dtype: int64

In [11]:
len(domain_2)

12096

### Baseline

In [12]:
X_1 = domain_1['sentence']
y_1 = domain_1['annotation']

X_2 = domain_2['sentence']
y_2 = domain_2['annotation']

In [13]:
# Vectorize text reviews to numbers
vec = CountVectorizer(stop_words='english')
X_1 = vec.fit_transform(X_1).toarray()
X_2 = vec.fit_transform(X_2).toarray()

In [14]:
print(X_1.shape)
print(X_2.shape)

print(y_1.shape)
print(y_2.shape)

(12096, 18638)
(12096, 15663)
(12096,)
(12096,)


In [15]:
new = [[0]]*len(X_1)

for i in range(len(X_1)):
    #new.append(X_1[i][:len(X_1[0])])
    new[i] = X_1[i][:15663]

In [16]:
X_1 = np.array(new)

In [17]:
print(X_1.shape)
print(X_2.shape)

print(y_1.shape)
print(y_2.shape)

(12096, 15663)
(12096, 15663)
(12096,)
(12096,)


### Train on Domain 1 and test on Domain 2

In [18]:
# training the model on training set from domain 1
model = MultinomialNB()
model.fit(X_1, y_1)

MultinomialNB()

In [19]:
# making predictions on the testing set from domain 2
y_pred = model.predict(X_2)

In [20]:
# compare predictions with true labels from domain 2
print("Naive Bayes model accuracy(in %):", metrics.accuracy_score(y_2, y_pred)*100)

Naive Bayes model accuracy(in %): 47.25529100529101


In [21]:
print(confusion_matrix(y_2, y_pred))

[[ 904 5144]
 [1236 4812]]


In [22]:
print(classification_report(y_2, y_pred))

              precision    recall  f1-score   support

    argument       0.42      0.15      0.22      6048
non_argument       0.48      0.80      0.60      6048

    accuracy                           0.47     12096
   macro avg       0.45      0.47      0.41     12096
weighted avg       0.45      0.47      0.41     12096



In [23]:
true_labels = y_2
predictions = y_pred

In [24]:
df_result = pd.DataFrame({'annotation': true_labels, 'predicted_labels_model_2_domain_2': predictions})

In [25]:
df_result.to_csv("M2_domain_2.csv", sep='\t', encoding='utf-8')

### Train on Domain 2 and test on Domain 1

In [26]:
# training the model on training set from domain 2
model = MultinomialNB()
model.fit(X_2, y_2)

MultinomialNB()

In [27]:
# making predictions on the testing set from domain 2
y_pred = model.predict(X_1)

In [28]:
# compare predictions with true labels from domain 1
print("Naive Bayes model accuracy(in %):", metrics.accuracy_score(y_1, y_pred)*100)

Naive Bayes model accuracy(in %): 46.44510582010582


In [29]:
print(confusion_matrix(y_1, y_pred))

[[ 791 5257]
 [1221 4827]]


In [30]:
print(classification_report(y_1, y_pred))

              precision    recall  f1-score   support

    argument       0.39      0.13      0.20      6048
non_argument       0.48      0.80      0.60      6048

    accuracy                           0.46     12096
   macro avg       0.44      0.46      0.40     12096
weighted avg       0.44      0.46      0.40     12096



In [31]:
true_labels = y_1
predictions = y_pred

In [32]:
df_result = pd.DataFrame({'annotation': true_labels, 'predicted_labels_model_2_domain_1': predictions})

In [33]:
df_result.to_csv("M2_domain_1.csv", sep='\t', encoding='utf-8')

In [34]:
df_result.head(3)

,annotation,predicted_labels_model_2_domain_1
3,non_argument,non_argument
4,non_argument,non_argument
5,non_argument,non_argument
